In [1]:
from verispy import VERIS

In [2]:
data_dir = 'C:/bharath/project/VCDB-master1/data/json/validated'
v = VERIS(json_dir=data_dir)

Found 8539 json files.


In [3]:
v.schema_url

'https://raw.githubusercontent.com/vz-risk/veris/master/verisc-merged.json'

In [4]:
df_veris = v.json_to_df(verbose=True)

Loading schema
Loading JSON files to DataFrame.


100%|█████████████████████████████████████████████████████████████████████████████| 8539/8539 [01:20<00:00, 105.86it/s]


Finished loading JSON files to dataframe.
Building DataFrame with enumerations.
Building enumeration columns.


100%|████████████████████████████████████████████████████████████████████████████████| 173/173 [00:23<00:00,  7.28it/s]


Done building DataFrame with enumerations.
Post-Processing DataFrame (A4 Names, Victim Industries, Patterns)
Finished building VERIS DataFrame


In [5]:
duplicateRowsDF = df_veris[df_veris.duplicated(['incident_id'])]
duplicateRowsDF['incident_id']

5       00163384-B4D7-46D5-9E6F-543DFB00F598
1203    18086BDC-29DD-46F7-8A05-A372C392CA95
2370    18086BDC-29DD-46F7-8A05-A372C392CA95
7019    d24d7280-6b0c-11e7-9b43-bf152d61c1c6
8072    ad2653d0-d3bd-11e8-9659-2bc8d207b0aa
Name: incident_id, dtype: object

In [10]:
df_veris.drop(8072,inplace = True)

In [81]:
df_veris['summary']

0       A billing clerk filed a claim for Patient A wi...
1       Patients at an Oregon healthcare facility were...
2       Jersey City Medical Center said a computer dis...
3       Sensitive information belonging to jobseekers ...
4       Veteran A returned a hard copy written prescri...
                              ...                        
8534    Willis North America recently began notifying ...
8535    Nearly a half-dozen former employees at Verizo...
8536    The government alleged that IRTS, which Frankl...
8537    A laptop containing personal and medical infor...
8538    Veteran A received Veteran B's medication from...
Name: summary, Length: 8534, dtype: object

In [11]:
df_veris[['asset.assets.variety.U - Mobile phone']].isnull().sum()

asset.assets.variety.U - Mobile phone    0
dtype: int64

In [78]:
df_veris.iloc[1595]['timeline.incident.year']

2015

In [48]:
df_common = df_veris.loc[[165,503,709,870,1023,1069,1114,1137,1214,1256,1259,1267,1441,1631,1670,1715,2095,2252,2329,
2362,2373,2463,2500,2533,2669,2805,2940,3047,3129,3182,3205,3209,3425,3720,3729,3875,3898,4020,
4108,4136,4267,4270,4358,4413,4430,4924,4981,4989,5049,5069,5177,5219,5310,5385,5498,5524,5533,
5601,5681,5944,6053,6210,6228,6254,6471,6582,6653,6692,6716,6782,6859,6865,7179,7331,7367,7377,
7380,7483,7490,7496,7626,7709,7935,8090,8115,8171,8451,8481]]

In [13]:
df_mobile = df_veris[df_veris['asset.assets.variety.U - Mobile phone']==True]

In [87]:
df_mobile[['reference','summary']].to_csv('mobilelinks1.csv')

In [21]:
df_social = df_veris[df_veris['action.social.vector.Social media']== True ]

In [22]:
df_mobile['action.Error'].value_counts()

False    115
True       6
Name: action.Error, dtype: int64

In [23]:
df_mobile.to_csv('mobilephone.csv')
df_social.to_csv('social.csv')

In [24]:
unigrams  = (
    df_mobile['summary'].str.lower()
                .str.replace(r'[^a-z\s]', '')
                .str.split(expand=True)
                .stack())
bigrams = unigrams + ' ' + unigrams.shift(-1)
trigrams = bigrams + ' ' + unigrams.shift(-2)

In [25]:
bigrams.to_csv('bigramsmobilephone.csv')

C:\Users\bhara\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [26]:
trigrams.to_csv('trigramsmobilephone.csv')

C:\Users\bhara\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [86]:
import xlsxwriter
veris_copy = df_common.copy()
summary = veris_copy[['summary']]
for col in veris_copy.columns:
    veris_copy[col] = veris_copy[col].apply(lambda x: col if x else '')
veris_copy['pattern'] = veris_copy.apply(lambda x: ','.join(x), axis = 1)
def get_first(pats):
    pats = [pat for pat in pats.split(',') if len(pat) > 0]
    return pats
veris_copy['pattern'] = veris_copy['pattern'].apply(lambda x: get_first(x))
common_excel = veris_copy[['pattern']]
import pandas as pd
list1 = list(veris_copy['pattern'])
res1 = list(set.intersection(*map(set, list1))) 
list2 = []
list3 = []
for i in list1:
    for j in i:
        if j not in res1:
            list2.append(j)
    list3.append(list2)
    list2 = []
#res1 = list(set.intersection(*map(set, list3)))
df1 = pd.Series(list3)
frame =  {'difference':df1}
df1 = pd.DataFrame(frame)
mobile_intersections = pd.concat([common_excel,df1,summary],ignore_index=True,axis=1, sort=False)
mobile_intersections.to_excel('mobile_intersections.xlsx')
#print(mobile_intersections)

In [28]:
veris_copy = df_mobile.copy()
for col in veris_copy.columns:
    veris_copy[col] = veris_copy[col].apply(lambda x: col if x else '')
veris_copy['pattern'] = veris_copy.apply(lambda x: ','.join(x), axis = 1)

In [29]:
veris_copy['pattern']

165     ,,action.Hacking,action.Malware,,,action.Socia...
503     ,,action.Hacking,action.Malware,,,action.Socia...
709     ,,action.Hacking,action.Malware,,,action.Socia...
870     ,,action.Hacking,action.Malware,,,action.Socia...
1023    ,,action.Hacking,action.Malware,,,action.Socia...
                              ...                        
8115    ,,action.Hacking,action.Malware,,,action.Socia...
8148    ,,,,action.Misuse,,,,action.environmental.note...
8171    ,,action.Hacking,action.Malware,,,action.Socia...
8451    ,,action.Hacking,action.Malware,,,action.Socia...
8481    ,,action.Hacking,action.Malware,,,action.Socia...
Name: pattern, Length: 121, dtype: object

In [30]:
def get_first(pats):
    pats = [pat for pat in pats.split(',') if len(pat) > 0]
    return pats
veris_copy['pattern'] = veris_copy['pattern'].apply(lambda x: get_first(x))

In [31]:
veris_copy['pattern']

165     [action.Hacking, action.Malware, action.Social...
503     [action.Hacking, action.Malware, action.Social...
709     [action.Hacking, action.Malware, action.Social...
870     [action.Hacking, action.Malware, action.Social...
1023    [action.Hacking, action.Malware, action.Social...
                              ...                        
8115    [action.Hacking, action.Malware, action.Social...
8148    [action.Misuse, action.environmental.notes, ac...
8171    [action.Hacking, action.Malware, action.Social...
8451    [action.Hacking, action.Malware, action.Social...
8481    [action.Hacking, action.Malware, action.Social...
Name: pattern, Length: 121, dtype: object

In [32]:
list1 = list(veris_copy['pattern'])
res = list(set.intersection(*map(set, list1))) 


C:\Users\bhara\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  if sys.path[0] == '':


In [44]:
import xlsxwriter   
workbook = xlsxwriter.Workbook('Frequent_Patterns_of_Mobile_Phones.xlsx') 
worksheet = workbook.add_worksheet() 
row = 0
column = 0
for item in res : 
  
    # write operation perform 
    worksheet.write(row, column, item) 
  
    # incrementing the value of row by one 
    # with each iteratons. 
    row += 1